In [7]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db

In [8]:
tau_d = 0.01
L_ls = 0.01
R_s = 0.01
L_lr = 0.01
R_r = 0.01
L_m = 1.0

K_p = L_ls/tau_d
K_i = R_s/tau_d
params_dict = {'L_m':L_m,
               'R_s':R_s,'L_ls':L_ls,
               'R_r':R_r,'L_lr':L_lr,               
               'N_pp':2.0,
               'J_r':1.0, 'B_r':0.1,
               'K_p':K_p,'K_i':K_i}


u_ini_dict = {'v_sd':0.0,'v_sq':325.0,'tau_l':0,'omega':2*np.pi*50}  # for the initialization problem
u_run_dict = {'v_sd':0.0,'v_sq':325.0,'tau_l':0,'omega':2*np.pi*50}  # for the running problem (here initialization and running problem are the same)


x_list = ['lambda_sd','lambda_sq','lambda_rd','lambda_rq','omega_r']    # [inductor current, PI integrator]
y_ini_list = ['tau_r'] # for the initialization problem
y_run_list = ['tau_r'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [9]:
# im:

L_s  =  L_m + L_ls
L_r  =  L_m + L_lr 
omega_e = N_pp*omega_r
i_sd = (L_m * lambda_rd - L_r * lambda_sd)/(L_m**2 - L_r * L_s)
i_sq = (L_m * lambda_rq - L_r * lambda_sq)/(L_m**2 - L_r * L_s)
i_rd = (L_m * lambda_sd - L_s * lambda_rd)/(L_m**2 - L_r * L_s)
i_rq = (L_m * lambda_sq - L_s * lambda_rq)/(L_m**2 - L_r * L_s)

dlambda_sd = v_sd - R_s * i_sd + omega * lambda_sq
dlambda_sq = v_sq - R_s * i_sq - omega * lambda_sd
dlambda_rd = -R_r * i_rd + (omega - omega_e) * lambda_rq 
dlambda_rq = -R_r * i_rq - (omega - omega_e) * lambda_rd

eq_tau_r = -tau_r + 3/2* N_pp * (lambda_sd *  i_sq - lambda_sq * i_sd)

# mech:
domega_r = 1/J_r*(tau_r - B_r*omega_r - tau_l)

In [5]:
f_list = [dlambda_sd,dlambda_sq,dlambda_rd,dlambda_rq,domega_r]
x_list = [ lambda_sd, lambda_sq, lambda_rd, lambda_rq, omega_r]
g_list = [eq_tau_r]
y_list = [   tau_r]

p_s = 3/2*(i_sq*v_sq + i_sd*v_sd)
q_s = 3/2*(i_sd*v_sq - i_sq*v_sd)

h_dict = {'i_sd':i_sd,'i_sq':i_sq,'i_rd':i_rd,'i_rq':i_rq,'i_pk':(i_sd**2+i_sq**2)**0.5,
          'tau_b':B_r*omega_r,'tau_l':tau_l,
         'p_s':p_s,'q_s':q_s}

sys = {'name':f"im",
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':h_dict
      }

sys = db.build(sys)


VerificationError: LinkError: command 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2019\\BuildTools\\VC\\Tools\\MSVC\\14.29.30133\\bin\\HostX86\\x64\\link.exe' failed with exit code 1104

In [6]:
f_list = [domega_r]
x_list = [omega_r]
g_list = [eq_tau_r,dlambda_sd,dlambda_sq,dlambda_rd,dlambda_rq]
y_list = [tau_r,lambda_sd,lambda_sq,lambda_rd,lambda_rq]
p_s = 3/2*(i_sq*v_sq + i_sd*v_sd)
q_s = 3/2*(i_sd*v_sq - i_sq*v_sd)

h_dict = {'i_sd':i_sd,'i_sq':i_sq,'i_rd':i_rd,'i_rq':i_rq,'i_pk':(i_sd**2+i_sq**2)**0.5,
          'tau_b':B_r*omega_r,'tau_l':tau_l,
         'p_s':p_s,'q_s':q_s}

sys = {'name':f"ima",
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':h_dict
      }

sys = db.build(sys)

Compilation time: 7.09 s


In [7]:
# ctrl:
epsilon_i_d =  i_d_ref - i_d
epsilon_i_q =  i_q_ref - i_q
u_d = K_p*epsilon_i_d + K_i*xi_i_d
u_q = K_p*epsilon_i_q + K_i*xi_i_q

eq_v_d = -u_d + v_d + L_s * omega_e * i_q 
eq_v_q = -u_q + v_q - L_s * omega_e * i_d -  omega_e * Phi 
dxi_i_d = epsilon_i_d
dxi_i_q = epsilon_i_q
eq_i_q_ref = - tau_r_ref + 3/2*Phi*N_pp*i_q_ref

Matrix([
[    -R_s/L_s,      omega_e, 0, 0,  0,                  i_q, 1/L_s,     0,                0],
[    -omega_e,     -R_s/L_s, 0, 0,  0, (-L_s*i_d - Phi)/L_s,     0, 1/L_s,                0],
[          -1,            0, 0, 0,  0,                    0,     0,     0,                0],
[           0,           -1, 0, 0,  0,                    0,     0,     0,                1],
[           0, 1.5*N_pp*Phi, 0, 0, -1,                    0,     0,     0,                0],
[           0,            0, 0, 0,  0,                   -1,     0,     0,                0],
[           0,  L_s*omega_e, 0, 0,  0,              L_s*i_q,     1,     0,                0],
[-L_s*omega_e,            0, 0, 0,  0,       -L_s*i_d - Phi,     0,     1,                0],
[           0,            0, 0, 0,  0,                    0,     0,     0, 1.5*N_pp*Phi + 1]])